In [48]:
from setup_notebook import setup_project_path
setup_project_path()

In [49]:
from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, MODELS_DIR

In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import  LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import cross_val_score

import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from datetime import datetime, date, timedelta
import os
import numpy as np
from scipy.stats import lognorm, loguniform, norm
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import cross_validate
import boto3
from io import StringIO
pd.set_option('display.max_columns',100)

## Load Data

In [51]:
data_dir = PROCESSED_DATA_DIR
models_dir = MODELS_DIR

os.makedirs(data_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

In [52]:
df_train_360 = pd.read_csv(os.path.join(data_dir,'KPIs_data_modelling_train_360_days.csv'))
df_test_360 = pd.read_csv(os.path.join(data_dir,'KPIs_data_modelling_test_360_days.csv'))

In [53]:
df_train_360[df_train_360.accounts_group=='2021-12_Uganda_SHS Entry-Level_Iganga_Sun King Home 120 EasyBuy']

,accounts_group,count_units,upfront_price_usd,avg_planned_repayment_days,frr_30,frr_60,frr_90,frr_180,frr_270,frr_360,frr_450,frr_540,frr_630,frr_720,repayment_speed_30,repayment_speed_60,repayment_speed_90,repayment_speed_180,repayment_speed_270,repayment_speed_360,repayment_speed_450,repayment_speed_540,repayment_speed_630,repayment_speed_720,avg_cum_days_disabled_30,avg_cum_days_disabled_60,avg_cum_days_disabled_90,avg_cum_days_disabled_180,avg_cum_days_disabled_270,avg_cum_days_disabled_360,avg_cum_days_disabled_450,avg_cum_days_disabled_540,avg_cum_days_disabled_630,avg_cum_days_disabled_720,at_risk_rate_30,at_risk_rate_60,at_risk_rate_90,at_risk_rate_180,at_risk_rate_270,at_risk_rate_360,at_risk_rate_450,at_risk_rate_540,at_risk_rate_630,at_risk_rate_720,disabled_gt_two_week_rate_30,disabled_gt_two_week_rate_60,disabled_gt_two_week_rate_90,disabled_gt_two_week_rate_180,disabled_gt_two_week_rate_270,disabled_gt_two_week_rate_360,disabled_gt_two_week_rate_450,disabled_gt_two_week_rate_540,disabled_gt_two_week_rate_630,disabled_gt_two_week_rate_720,unlocked_rate_30,unlocked_rate_60,unlocked_rate_90,unlocked_rate_180,unlocked_rate_270,unlocked_rate_360,unlocked_rate_450,unlocked_rate_540,unlocked_rate_630,unlocked_rate_720,disabled_rate_30,disabled_rate_60,disabled_rate_90,disabled_rate_180,disabled_rate_270,disabled_rate_360,disabled_rate_450,disabled_rate_540,disabled_rate_630,disabled_rate_720,frr_3_years,actual_fr,total_follow_on_revenue_usd,reg_month,country,product_group,area,primary_product,age_in_months,cohort_age
39163,2021-12_Uganda_SHS Entry-Level_Iganga_Sun King...,7,135.85894,371,0.156767,0.261685,0.33593,0.513145,0.711295,0.848832,0.903846,0.93038,-99.0,-99.0,2.4816,1.7976,1.4735,1.0799,0.9846,0.8754,0.9038,0.9303,-99.0,-99.0,0.0,4.0,11.0,21.0,34.0,40.0,60.0,68.0,-99.0,-99.0,0.142857,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,-99.0,-99.0,0.0,0.142857,0.142857,0.142857,0.285714,0.428571,0.714286,0.857143,-99.0,-99.0,0.285714,0.285714,0.0,0.142857,0.0,0.142857,0.285714,0.142857,-99.0,-99.0,0.991967,1146.222065,1155.50423,2021-12,Uganda,SHS Entry-Level,Iganga,Sun King Home 120 EasyBuy,44,540


In [54]:
df_train_360.accounts_group.sample(frac=0.001).tolist()

['2021-12_Kenya_SHS with TV_Kendu Bay_Sun King Home 600 Easybuy GSM',
 '2018-10_Uganda_SHS Entry-Level_Mityana_Sun King Home 120 EasyBuy',
 '2021-10_Kenya_SHS Entry-Level_Molo_Sun King Home 120 EasyBuy',
 '2020-06_Myanmar (Burma)_SHS with TV_Pyay_Sun King Home 400 Easybuy GSM',
 '2021-09_Kenya_SHS Entry-Level Upgrade_Kehancha_Sun King Home 60 EasyBuy',
 '2020-12_Tanzania_SHS with TV Upgrade_Uyole_Sun King Home 400 Easybuy GSM',
 '2020-09_Nigeria_SHS Entry-Level_Benin_Sun King Home 60 EasyBuy',
 '2021-06_Tanzania_Lanterns_Mafinga_Sun King Pro EasyBuy',
 '2020-09_Kenya_Lanterns_Kisumu_Sun King Pro EasyBuy',
 '2020-01_Uganda_Lanterns_Mpigi_Sun King Pro EasyBuy',
 '2021-06_Kenya_SHS with TV_Nyeri_Sun King Home 400 Easybuy GSM',
 '2020-05_Kenya_SHS Entry-Level_Kiritiri_Sun King Home 120 Plus EasyBuy',
 '2021-10_Kenya_SHS with TV_Marsabit_Sun King Home 400 Easybuy GSM',
 '2020-01_Kenya_Lanterns_Kipkaren River_Sun King Boom EasyBuy',
 '2021-05_Nigeria_SHS Entry-Level_Ijebu Ode_Sun King Home 6

In [55]:
df_test_360.shape

(8987, 84)

In [56]:
df_train_360.shape

(50928, 84)

In [57]:
df_train_360

,accounts_group,count_units,upfront_price_usd,avg_planned_repayment_days,frr_30,frr_60,frr_90,frr_180,frr_270,frr_360,frr_450,frr_540,frr_630,frr_720,repayment_speed_30,repayment_speed_60,repayment_speed_90,repayment_speed_180,repayment_speed_270,repayment_speed_360,repayment_speed_450,repayment_speed_540,repayment_speed_630,repayment_speed_720,avg_cum_days_disabled_30,avg_cum_days_disabled_60,avg_cum_days_disabled_90,avg_cum_days_disabled_180,avg_cum_days_disabled_270,avg_cum_days_disabled_360,avg_cum_days_disabled_450,avg_cum_days_disabled_540,avg_cum_days_disabled_630,avg_cum_days_disabled_720,at_risk_rate_30,at_risk_rate_60,at_risk_rate_90,at_risk_rate_180,at_risk_rate_270,at_risk_rate_360,at_risk_rate_450,at_risk_rate_540,at_risk_rate_630,at_risk_rate_720,disabled_gt_two_week_rate_30,disabled_gt_two_week_rate_60,disabled_gt_two_week_rate_90,disabled_gt_two_week_rate_180,disabled_gt_two_week_rate_270,disabled_gt_two_week_rate_360,disabled_gt_two_week_rate_450,disabled_gt_two_week_rate_540,disabled_gt_two_week_rate_630,disabled_gt_two_week_rate_720,unlocked_rate_30,unlocked_rate_60,unlocked_rate_90,unlocked_rate_180,unlocked_rate_270,unlocked_rate_360,unlocked_rate_450,unlocked_rate_540,unlocked_rate_630,unlocked_rate_720,disabled_rate_30,disabled_rate_60,disabled_rate_90,disabled_rate_180,disabled_rate_270,disabled_rate_360,disabled_rate_450,disabled_rate_540,disabled_rate_630,disabled_rate_720,frr_3_years,actual_fr,total_follow_on_revenue_usd,reg_month,country,product_group,area,primary_product,age_in_months,cohort_age
0,2020-10_Myanmar (Burma)_SHS Entry-Level_Myaung...,3,46.331370,560,0.082418,0.119880,0.173077,0.324509,0.456710,0.537463,0.646354,0.753497,0.821678,0.821678,2.8125,1.4229,1.2434,1.0673,0.9740,0.8481,0.8094,0.7821,0.8216,0.8216,0.0,0.0,0.0,0.0,8.0,8.0,15.0,17.0,19.0,19.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.666667,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.333333,0.666667,0.000000,0.333333,0.821678,381.075371,463.777016,2020-10,Myanmar (Burma),SHS Entry-Level,Myaungmya,Sun King Home 120 EasyBuy,58,720
1,2019-10_Nigeria_SHS Entry-Level_Akure_Sun King...,26,790.666812,280,0.095830,0.203945,0.278924,0.551395,0.778586,0.854945,0.889884,0.891012,0.892139,0.894393,1.1374,1.0505,0.9174,0.8701,0.8081,0.8549,0.8898,0.8910,0.8921,0.8943,0.0,4.0,6.0,21.0,37.0,47.0,55.0,55.0,55.0,64.0,0.038462,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.153846,0.153846,0.192308,0.192308,0.192308,0.192308,0.153846,0.000000,0.000000,0.000000,0.038462,0.192308,0.692308,0.769231,0.807692,0.807692,0.807692,0.115385,0.115385,0.192308,0.230769,0.230769,0.230769,0.192308,0.192308,0.192308,0.153846,0.913666,4482.197041,4905.728178,2019-10,Nigeria,SHS Entry-Level,Akure,Sun King Home 120 EasyBuy,70,720
2,2021-11_Kenya_SHS with TV_Maua_Sun King Home 4...,4,214.301564,714,0.022537,0.053294,0.074506,0.168190,0.276370,0.365812,0.453486,0.527020,0.604442,0.686107,0.6929,0.7110,0.6347,0.6875,0.7431,0.7328,0.7238,0.6992,0.6860,0.7008,4.0,13.0,28.0,54.0,68.0,88.0,121.0,159.0,194.0,220.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.500000,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.250000,0.250000,0.000000,0.924321,2334.611385,2525.758243,2021-11,Kenya,SHS with TV,Maua,Sun King Home 400 Easybuy GSM,45,720
3,2020-10_Kenya_Lanterns_Luanda_Sun King Boom Ea...,565,2599.824900,287,0.078688,0.153720,0.224591,0.419354,0.591209,0.718745,0.788589,0.836033,0.860914,0.878785,0.9579,0.8121,0.7576,0.6787,0.6294,0.7187,0.7885,0.8360,0.

## More Feature Engineering

In [58]:
def feature_engineering(df_train, df_test):
    
    # Creating column for Unlock price
    df_train['unlock_price_usd'] = df_train['upfront_price_usd'] + df_train['total_follow_on_revenue_usd']
    df_test['unlock_price_usd'] = df_test['upfront_price_usd'] + df_test['total_follow_on_revenue_usd']
    
    # Calculating averge unlock and upfront price

    df_train['avg_upfront_price_usd'] = np.round((df_train['upfront_price_usd']/df_train['count_units']),0)
    df_train['avg_unlock_price_usd'] = np.round((df_train['unlock_price_usd']/df_train['count_units']),0)

    df_test['avg_upfront_price_usd'] = np.round((df_test['upfront_price_usd']/df_test['count_units']),0)
    df_test['avg_unlock_price_usd'] = np.round((df_test['unlock_price_usd']/df_test['count_units']),0)
    
    # Removing unnecessary columns
    #cols_to_remove = [col for col in df_train.columns if ('repayment_speed_' in col)]
    cols_to_remove = []
    cols_to_remove.append('upfront_price_usd')
    cols_to_remove.append('unlock_price_usd')

    print('columns to remove: ',cols_to_remove)
    
    df_train.drop(cols_to_remove, axis=1, inplace=True)
    df_test.drop(cols_to_remove, axis=1, inplace=True)
    
    print('Sample from train data------------')
    print(df_train.sample(1))
    
    print('Sample from test data------------')
    print(df_test.sample(1))

    df_train = df_train.reset_index().drop(columns = 'index')
    df_test = df_test.reset_index().drop(columns = 'index')
    
    return df_train, df_test

In [59]:
df_train, df_test = feature_engineering(df_train_360, df_test_360)

columns to remove:  ['upfront_price_usd', 'unlock_price_usd']
Sample from train data------------
                                          accounts_group  count_units  \
11519  2020-12_Kenya_SHS with TV_Mutomo_Sun King Home...            1   

       avg_planned_repayment_days    frr_30    frr_60    frr_90   frr_180  \
11519                         553  0.040389  0.094371  0.130422  0.202911   

        frr_270   frr_360   frr_450   frr_540   frr_630   frr_720  \
11519  0.226046  0.256796  0.295257  0.323983  0.358878  0.364662   

       repayment_speed_30  repayment_speed_60  repayment_speed_90  \
11519              0.9588              0.9721              0.8579   

       repayment_speed_180  repayment_speed_270  repayment_speed_360  \
11519               0.6404               0.4692               0.3971   

       repayment_speed_450  repayment_speed_540  repayment_speed_630  \
11519               0.3639               0.3318               0.3588   

       repayment_speed_720  avg_c

## Data Splitting

In [60]:
x_train = df_train.drop(['accounts_group','count_units','frr_3_years','total_follow_on_revenue_usd',
                       'reg_month','country','product_group','area','primary_product','age_in_months',
                      'actual_fr', 'cohort_age'], axis=1)
y_train = df_train[:]['frr_3_years']

print('shape of the input data: ',x_train.shape)
print('shape of the output data: ',y_train.shape)

x_test = df_test.drop(['accounts_group','count_units','frr_3_years','total_follow_on_revenue_usd',
                       'reg_month','country','product_group','area','primary_product','age_in_months',
                      'actual_fr', 'cohort_age'], axis=1)
y_test = df_test[:]['frr_3_years']

print('shape of the input test data: ',x_test.shape)
print('shape of the output test data: ',y_test.shape)

shape of the input data:  (50928, 73)
shape of the output data:  (50928,)
shape of the input test data:  (8987, 73)
shape of the output test data:  (8987,)


In [61]:
x_train.columns

Index(['avg_planned_repayment_days', 'frr_30', 'frr_60', 'frr_90', 'frr_180',
       'frr_270', 'frr_360', 'frr_450', 'frr_540', 'frr_630', 'frr_720',
       'repayment_speed_30', 'repayment_speed_60', 'repayment_speed_90',
       'repayment_speed_180', 'repayment_speed_270', 'repayment_speed_360',
       'repayment_speed_450', 'repayment_speed_540', 'repayment_speed_630',
       'repayment_speed_720', 'avg_cum_days_disabled_30',
       'avg_cum_days_disabled_60', 'avg_cum_days_disabled_90',
       'avg_cum_days_disabled_180', 'avg_cum_days_disabled_270',
       'avg_cum_days_disabled_360', 'avg_cum_days_disabled_450',
       'avg_cum_days_disabled_540', 'avg_cum_days_disabled_630',
       'avg_cum_days_disabled_720', 'at_risk_rate_30', 'at_risk_rate_60',
       'at_risk_rate_90', 'at_risk_rate_180', 'at_risk_rate_270',
       'at_risk_rate_360', 'at_risk_rate_450', 'at_risk_rate_540',
       'at_risk_rate_630', 'at_risk_rate_720', 'disabled_gt_two_week_rate_30',
       'disabled_gt_tw

## Tuning

In [62]:
x_test.to_csv("../data/x_test.csv" ,index=False)
y_test.to_csv("../data/y_test.csv" ,index=False)
x_train.to_csv("../data/x_train.csv" ,index=False)
y_train.to_csv("../data/y_train.csv" ,index=False)

In [63]:
n_estimators = [50,100,150,200,250,350,450,500,1000,1500]

space = {
    'num_leaves': hp.loguniform("num_leaves", np.log(5), np.log(200)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1.5)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 250, 200000, 50),
    'min_child_weight': hp.loguniform('min_child_weight', np.log(0.00001), np.log(1)),
    'min_child_samples': hp.quniform('min_child_samples', 0, 200, 5),
    'n_estimators': hp.choice("n_estimators", n_estimators),
    'max_depth': hp.loguniform('max_depth', np.log(3), np.log(50)),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0.001, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0.1, 1),
    'subsample': hp.uniform('subsample', 0.1, 1)
}

def hyperparameter_tuning(space):
    
    model = LGBMRegressor(
        n_estimators=int(space['n_estimators']),
        num_leaves=int(space['num_leaves']),
        learning_rate=space['learning_rate'],
        max_depth=int(space['max_depth']),
        subsample_for_bin=int(space['subsample_for_bin']),
        min_child_weight=space['min_child_weight'],
        min_child_samples=int(space['min_child_samples']),
        colsample_bytree=space['colsample_bytree'],
        reg_alpha=space['reg_alpha'],
        reg_lambda=space['reg_lambda'],
        subsample=space['subsample'],
        objective="quantile",
        alpha=0.5,
        n_jobs=-1,
        verbose=-1,
        random_state=1234
    )

    model.set_params(eval_set=[(x_test, y_test)], eval_metric='quantile')

    score_cv = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()
    
    return {'loss': -score_cv, 'status': STATUS_OK, 'model': model}

# trials = Trials()
# best_pos_rmse = fmin(fn=hyperparameter_tuning,
#                      space=space,
#                      algo=tpe.suggest,
#                      max_evals=100,
#                      trials=trials)

# print(best_pos_rmse)

In [64]:
best_pos_rmse = {'colsample_bytree': np.float64(0.47944292609970934),
 'learning_rate': np.float64(0.05409326240495583),
 'max_depth': np.float64(12.28243840861942),
 'min_child_samples': np.float64(30.0),
 'min_child_weight': np.float64(0.004744862455523174),
 'n_estimators': 1000,
 'num_leaves': np.float64(172.376201928432),
 'reg_alpha': np.float64(0.5410761578424548),
 'reg_lambda': np.float64(0.8763999421949613),
 'subsample': np.float64(0.38357297478931585),
 'objective': 'quantile',
 'subsample_for_bin': np.float64(10050.0),
"alpha": 0.5,
"n_jobs": -1,
"random_state": 42,
"verbose": -1
 }


# best_pos_rmse = {'colsample_bytree': np.float64(0.6298988695293308), 'learning_rate': np.float64(0.007302754185728239), 'max_depth': np.float64(10.0), 'min_child_samples': np.float64(15.0), 'min_child_weight': np.float64(0.016037933648002045), 'n_estimators': np.float64(1700.0), 'num_leaves': np.float64(85.0), 'reg_alpha': np.float64(0.8573918645599341), 'reg_lambda': np.float64(0.600743674332251), 'subsample': np.float64(0.8975495771174369), 'subsample_for_bin': np.float64(58000.0)}

# best_pos_rmse = {"n_estimators": 750, "num_leaves": 199, "learning_rate": 0.11284191380100693, "max_depth": 9, "min_child_weight": 0.011338492995747648, "min_child_samples": 170, "colsample_bytree": 0.5567982554608459, "reg_alpha": 0.5734609694174653, "reg_lambda": 0.05407787444418399, "subsample": 0.8856904087979187, "objective": "quantile", "alpha": 0.5, "n_jobs": -1, "random_state": 42, "verbose": -1}
# best_pos_rmse = {'colsample_bytree': np.float64(0.6419407898437077), 'learning_rate': np.float64(0.024693044008451875), 'max_depth': np.float64(14.0), 'min_child_samples': np.float64(110.0), 'min_child_weight': np.float64(0.47146317735184073), 'n_estimators': np.float64(1650.0), 'num_leaves': np.float64(119.0), 'reg_alpha': np.float64(0.15717970056189712), 'reg_lambda': np.float64(0.6176608512730031), 'subsample': np.float64(0.536830512725262), 'subsample_for_bin': np.float64(46000.0)}

best_pos_rmse = {
            "n_estimators": 1650,
            "num_leaves": 119,
            "learning_rate": 0.024693044008451875,
            "max_depth": 14,
            "min_child_weight": 0.47146317735184073,
            "min_child_samples": 110,
            "colsample_bytree": 0.6419407898437077,
            "reg_alpha": 0.15717970056189712,
            "reg_lambda": 0.6176608512730031,
            "subsample": 0.536830512725262,
            "objective": "quantile",
            "alpha": 0.5,
            "n_jobs": -1,
            "random_state": 42,
            "verbose": -1
        }

In [65]:
#getting the best params

# int_params = ['max_depth','min_child_samples','num_leaves','subsample_for_bin']
int_params = ["max_depth", "min_child_samples", "num_leaves", "subsample_for_bin", "n_estimators"]
best_pos_rmse_params = {k: int(v) if k in int_params else v for k,v in best_pos_rmse.items()}
print(best_pos_rmse_params)


{'n_estimators': 1650, 'num_leaves': 119, 'learning_rate': 0.024693044008451875, 'max_depth': 14, 'min_child_weight': 0.47146317735184073, 'min_child_samples': 110, 'colsample_bytree': 0.6419407898437077, 'reg_alpha': 0.15717970056189712, 'reg_lambda': 0.6176608512730031, 'subsample': 0.536830512725262, 'objective': 'quantile', 'alpha': 0.5, 'n_jobs': -1, 'random_state': 42, 'verbose': -1}


In [66]:
model_reg_lgbm_qntl_tuned_pos_rmse = LGBMRegressor(boosting_type='gbdt', class_weight=None, 
                                                   device_type='cpu', importance_type='split', 
                                                   num_threads=-1,
                                                   **best_pos_rmse_params)

model_reg_lgbm_qntl_tuned_pos_rmse.fit(x_train,y_train, eval_set=(x_test,y_test), eval_metric='quantile')

print(model_reg_lgbm_qntl_tuned_pos_rmse.best_iteration_)
print(model_reg_lgbm_qntl_tuned_pos_rmse.best_score_)
print(model_reg_lgbm_qntl_tuned_pos_rmse.get_params)
print(model_reg_lgbm_qntl_tuned_pos_rmse)
#print(math.sqrt(mean_squared_error(y_val,model_reg_lgbm_qntl_tuned_pos_rmse.predict(x_val))))
#print(math.sqrt(mean_squared_error(y_val_oot,model_reg_lgbm_qntl_tuned_pos_rmse.predict(x_val_oot))))

0
defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict({'quantile': 0.012055570038646624})})
<bound method LGBMModel.get_params of LGBMRegressor(alpha=0.5, colsample_bytree=0.6419407898437077, device_type='cpu',
              learning_rate=0.024693044008451875, max_depth=14,
              min_child_samples=110, min_child_weight=0.47146317735184073,
              n_estimators=1650, n_jobs=-1, num_leaves=119, num_threads=-1,
              objective='quantile', random_state=42,
              reg_alpha=0.15717970056189712, reg_lambda=0.6176608512730031,
              subsample=0.536830512725262, verbose=-1)>
LGBMRegressor(alpha=0.5, colsample_bytree=0.6419407898437077, device_type='cpu',
              learning_rate=0.024693044008451875, max_depth=14,
              min_child_samples=110, min_child_weight=0.47146317735184073,
              n_estimators=1650, n_jobs=-1, num_leaves=119, num_threads=-1,
              objective='quantile', random_state=42,
              reg_alp

In [67]:
test_preds = model_reg_lgbm_qntl_tuned_pos_rmse.predict(x_test)

In [68]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_pinball_loss
results = {
    "rmse": math.sqrt(mean_squared_error(y_test,test_preds)),
    "mae": mean_absolute_error(y_test, test_preds),
    "mape": np.mean(np.abs((y_test - test_preds) / y_test)),
    "r2": r2_score(y_test, test_preds),
    "qloss":  mean_pinball_loss(y_test, test_preds, alpha=0.5)
}

results

{'rmse': 0.05191260019209819,
 'mae': 0.024111140054557622,
 'mape': 0.04234076165899468,
 'r2': 0.9173584555031005,
 'qloss': 0.012055570027278811}

In [69]:
def save_model(model_name, model):
    model_file_name = "quantile_regression_"+model_name+"_days_"+ str(date.today()) +".txt"
    model.booster_.save_model(os.path.join(models_dir, model_file_name))

In [70]:
save_model('360_'+str(0.5), model_reg_lgbm_qntl_tuned_pos_rmse)

In [71]:
import lightgbm

In [72]:
model = LGBMRegressor(alpha=0.5, boosting_type='gbdt', class_weight=None, 
                                                   device_type='cpu', importance_type='split',n_jobs=-1, 
                                                   num_threads=-1, objective='quantile',random_state=1234,
                                                verbose=-1)

model.fit(x_train,y_train, eval_set=(x_test,y_test), eval_metric='quantile')

test_preds = model.predict(x_test)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_pinball_loss
results = {
    "rmse": math.sqrt(mean_squared_error(y_test,test_preds)),
    "mae": mean_absolute_error(y_test, test_preds),
    "mape": np.mean(np.abs((y_test - test_preds) / y_test)),
    "r2": r2_score(y_test, test_preds),
    "qloss":  mean_pinball_loss(y_test, test_preds, alpha=0.5)
}

results

{'rmse': 0.056887149041923886,
 'mae': 0.028123331213945897,
 'mape': 0.04843192657186717,
 'r2': 0.900761269737798,
 'qloss': 0.014061665606972949}